In [ ]:
import columns
import paths

# Concat INMET files

In [ ]:
import pandas as pd
import os

desired_columns = [
    columns.unix_date_time_column,
    'PRECIPITACAO TOTAL, HORARIO (mm)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
    'RADIACAO GLOBAL (KJ/m2)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (C)',
    'TEMPERATURA DO PONTO DE ORVALHO (C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'VENTO, DIRECAO HORARIA (gr) ( (gr))',
    'VENTO, RAJADA MAXIMA (m/s)',
    'VENTO, VELOCIDADE HORARIA (m/s)'
]

def get_date_column(df) -> str:
    for col in ['DATA (YYYY-MM-DD)', 'Data']:
        if col in df.columns:
            return col
    raise KeyError('No date column found')

def get_hour_column(df) -> str:
    for col in ['HORA (UTC)', 'Hora UTC']:
        if col in df.columns:
            return col
    raise KeyError('No hour column found')


is_first_write = True
stations_df = pd.DataFrame()
for root, _, files in os.walk(paths.raw_inmet_folder):  # Navigate in every csv inside the folder and read it
    for file in files:
        if not file.lower().endswith('.csv'):
            continue

        file_path = os.path.join(root, file)
        with open(file_path, 'r') as f:
            station_dict = {}  # Get the first 8 lines as station metadata
            for line in f.readlines()[:8]:
                key, value = line.strip().split(':;', 1)
                station_dict[key.strip()] = value.strip()
            stations_df = pd.concat([stations_df, pd.DataFrame([station_dict])], ignore_index=True)

        curr_df = pd.read_csv(file_path, skiprows=8, sep=';', decimal=',').dropna(axis=1, how='all')  # Read the csv skipping the first 8 rows and dropping empty columns
        curr_df[columns.unix_date_time_column] = pd.to_datetime(curr_df[get_date_column(curr_df)] + ' ' + curr_df[get_hour_column(curr_df)], format='mixed', utc=True, errors='coerce').astype(int) // 10**9 // 3600
        try:
            curr_df = curr_df[desired_columns]  # Keep only the desired columns
        except KeyError as ke:
            continue  # Skip files that do not have all the desired columns
        curr_df.replace(-9999.0, pd.NA, inplace=True)
        curr_df.dropna(inplace=True)
        curr_df[columns.code_meta_label] = station_dict[columns.code_meta_label]  # Add the station code to the dataframe

        curr_df.to_csv(paths.inmet_concat_file, mode='w' if is_first_write else 'a', header=is_first_write, index=False)
        is_first_write = False

stations_df = stations_df = ( # Keeps the first non-null value for each column among duplicates
    stations_df
    .groupby(columns.code_meta_label, as_index=False)
    .agg(lambda x: x.dropna().iloc[0] if x.dropna().any() else pd.NA)
)
stations_df = stations_df.loc[:, ~stations_df.columns.str.contains(r'\?|\bDATA\b', regex=True, case=False)]  # Remove columns with '?' or '\bDATA\b' in the name
numeric_columns = ['LATITUDE', 'LONGITUDE', 'ALTITUDE']
stations_df[numeric_columns] = stations_df[numeric_columns].replace(',',  '.', regex=True).apply(pd.to_numeric, errors='coerce')
stations_df.to_csv(paths.inmet_stations_file, mode='w', index=False)

# Concat INPE All Satellites files

In [ ]:
import pandas as pd
import os

date_column = 'data_pas'

desired_columns = ['latitude', 'longitude', columns.unix_date_time_column, 'frp']

is_first_write = True
for root, _, files in os.walk(paths.raw_inpe_all_folder):  # Navigate in every csv inside the folder and read it
    for file in files:
        if not file.lower().endswith('.csv'):
            continue

        file_path = os.path.join(root, file)
        curr_df = pd.read_csv(file_path, sep=',', decimal='.')

        # Add unix date time column rounded to the nearest hour
        curr_df[columns.unix_date_time_column] = pd.to_datetime(curr_df[date_column], utc=True).dt.round('H').astype(int) // 10**9 // 3600

        try:
            curr_df = curr_df[desired_columns]  # Keep only the desired columns
        except KeyError as ke:
            continue  # Skip files that do not have all the desired columns
        curr_df.replace(-999.0, pd.NA, inplace=True)
        curr_df.dropna(inplace=True)

        curr_df.to_csv(paths.inpe_all_concat_file, mode='w' if is_first_write else 'a', header=is_first_write, index=False)
        is_first_write = False